In [1]:
import pandas as pd
import os
import re

In [2]:
# Loading TISdb data
df_tisdb = pd.read_csv('data_raw/human_tisdb_data_1.0.csv.gz', compression='gzip')
df_tisdb

,Gene,Transcript,Chr,Strand,Coordinate start codon,Start Codon,Coordinate stop codon,Stop Codon,TIS type,Predicted uORF
0,RANGRF,NM_001177802,chr17,+,8192082,ATG,8192175,TGA,uTIS,No
1,RANGRF,NM_016492,chr17,+,8192082,ATG,8192175,TGA,uTIS,No
2,RANGRF,NM_001177801,chr17,+,8192082,ATG,8192175,TGA,uTIS,No
3,HIST1H4B,NM_003544,chr6,-,26027228,ATG,26027170,TAA,dTIS,No
4,FGFR1OP2,NM_001171887,chr12,+,27107092,ATG,27117664,TGA,aTIS,No
...,...,...,...,...,...,...,...,...,...,...
11918,SERPINH1,NM_001235,chr11,+,75273335,ACG,75273347,TAG,uTIS,Yes
11919,SERPINH1,NM_001207014,chr11,+,75273310,GTG,75274774,TGA,uTIS,Yes
11920,SERPINH1,NM_001235,chr11,+,75273310,GTG,75277456,TGA,uTIS,No
11921,C5orf54,NM_022090,chr5,-,159827012,ATG,159826915,TAG,uTIS,Yes


In [3]:
# Loading Transcirc metadata
df_transcirc = pd.read_csv('data_raw/transcirc_metadata.tsv.gz', sep='\t', compression='gzip')

# Selecting circRNAs with TIS annotation
df_transcirc_tis = df_transcirc[df_transcirc['TIS'] == 1.0]
df_transcirc_tis

,TransCirc_ID,xref,species,gene,strand,gene_id,chrom,start,end,evidences_num,evidences_score,MS_Qvalue,m6A,RP/RP,TIS,ORF,IRES,Peptide composition,MS
11,TC-hsa-RP1-283E3_0004,circAtlas:hsa-RP1-283E3_0015; circpedia2:HSA_C...,Homo sapiens,RP1-283E3,-,ENSG00000227775.3,chr1,1716346,1722831,2,1.396139,NaN,0.0,0.0,1.0,0.0,0.396139,0.000000,0.0
12,TC-hsa-RP1-283E3_0006,circAtlas:hsa-RP1-283E3_0009; circpedia2:HSA_C...,Homo sapiens,RP1-283E3,-,ENSG00000227775.3,chr1,1719358,1722831,4,3.191850,NaN,0.0,0.0,1.0,1.0,0.374504,0.817346,0.0
29,TC-hsa-GNB1_0028,circAtlas:hsa-GNB1_0011; circBase:hsa_circ_000...,Homo sapiens,GNB1,-,ENSG00000078369.17,chr1,1793245,1839238,2,1.227430,NaN,0.0,0.0,1.0,0.0,0.227430,0.000000,0.0
30,TC-hsa-GNB1_0038,circAtlas:hsa-GNB1_0029,Homo sapiens,GNB1,-,ENSG00000078369.17,chr1,1804419,1848036,4,2.976764,NaN,0.0,0.0,1.0,1.0,0.227430,0.749334,0.0
31,TC-hsa-GNB1_0041,circAtlas:hsa-GNB1_0062,Homo sapiens,GNB1,-,ENSG00000078369.17,chr1,1804419,1879902,4,2.976764,NaN,0.0,0.0,1.0,1.0,0.227430,0.749334,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
327799,TC-hsa-ARHGEF6_0026,circAtlas:hsa-ARHGEF6_0042; circpedia2:HSA_CIR...,Homo sapiens,ARHGEF6,-,ENSG00000129675.15,chrX,136743585,136745347,4,2.982721,NaN,0.0,0.0,1.0,1.0,0.292489,0.690231,0.0
327800,TC-hsa-ARHGEF6_0027,circAtlas:hsa-ARHGEF6_0004; deepbase2:hsa-circ...,Homo sapiens,ARHGEF6,-,ENSG00000129675.15,chrX,136743585,136747592,4,2.951277,NaN,0.0,0.0,1.0,1.0,0.305866,0.645411,0.0
327801,TC-hsa-ARHGEF6_0028,circAtlas:hsa-ARHGEF6_0017,Homo sapiens,ARHGEF6,-,ENSG00000129675.15,chrX,136743585,136781149,4,3.131483,NaN,0.0,0.0,1.0,1.0,0.428511,0.702972,0.0
328063,TC-hsa-CXorf40A_0001,circAtlas:hsa-CXorf40A_0002,Homo sapiens,CXorf40A,+,ENSG00000197620.10,chrX,149542019,149546917,3,2.258650,NaN,1.0,0.0,1.0,0.0,0.258650,0.000000,0.0


In [4]:
df_map_1 = pd.read_csv('data_raw/circRNA_TIS_1.tsv', sep='\t')
df_map_1

,transcirc_id,circ_strand,circ_chrom,tis_coordinate_hg38,tis_position_in_circrna
0,TC-hsa-GNB1_0028,-,chr1,1825453,47
1,TC-hsa-GNB1_0038,-,chr1,1825453,47
2,TC-hsa-GNB1_0041,-,chr1,1825453,47
3,TC-hsa-GNB1_0076,-,chr1,1825453,47
4,TC-hsa-GNB1_0065,-,chr1,1825453,47
...,...,...,...,...,...
9686,TC-hsa-ARHGEF6_0026,-,chrX,136745328,20
9687,TC-hsa-ARHGEF6_0027,-,chrX,136745328,105
9688,TC-hsa-ARHGEF6_0028,-,chrX,136745328,621
9689,TC-hsa-CXorf40A_0001,+,chrX,149545631,211


In [5]:
df_map_1['transcirc_id'].nunique()

8500

In [6]:
df_not_mapped = pd.merge(df_transcirc_tis, df_map_1, left_on='TransCirc_ID', right_on='transcirc_id', how='left')
df_not_mapped = df_not_mapped[df_not_mapped['transcirc_id'].isna()]
df_not_mapped

,TransCirc_ID,xref,species,gene,strand,gene_id,chrom,start,end,evidences_num,...,TIS,ORF,IRES,Peptide composition,MS,transcirc_id,circ_strand,circ_chrom,tis_coordinate_hg38,tis_position_in_circrna
0,TC-hsa-RP1-283E3_0004,circAtlas:hsa-RP1-283E3_0015; circpedia2:HSA_C...,Homo sapiens,RP1-283E3,-,ENSG00000227775.3,chr1,1716346,1722831,2,...,1.0,0.0,0.396139,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
1,TC-hsa-RP1-283E3_0006,circAtlas:hsa-RP1-283E3_0009; circpedia2:HSA_C...,Homo sapiens,RP1-283E3,-,ENSG00000227775.3,chr1,1719358,1722831,4,...,1.0,1.0,0.374504,0.817346,0.0,NaN,NaN,NaN,NaN,NaN
35,TC-hsa-RP1-283E3_0007,circAtlas:hsa-RP1-283E3_0017; circpedia2:HSA_C...,Homo sapiens,RP1-283E3,-,ENSG00000227775.3,chr1,1721596,1722831,2,...,1.0,0.0,0.329381,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
61,TC-hsa-RP1-283E3_0008,circAtlas:hsa-RP1-283E3_0032,Homo sapiens,RP1-283E3,-,ENSG00000227775.3,chr1,1721596,1734835,4,...,1.0,1.0,0.329381,0.735423,0.0,NaN,NaN,NaN,NaN,NaN
75,TC-hsa-CLIC4_0003,circAtlas:hsa-CLIC4_0038,Homo sapiens,CLIC4,+,ENSG00000169504.14,chr1,24791464,24827116,4,...,1.0,1.0,0.318250,0.780175,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10538,TC-hsa-PLS3_0018,circAtlas:hsa-PLS3_0050; circpedia2:HSA_CIRCpe...,Homo sapiens,PLS3,+,ENSG00000102024.17,chrX,115629198,115629967,4,...,1.0,1.0,0.275340,0.805254,0.0,NaN,NaN,NaN,NaN,NaN
10539,TC-hsa-PLS3_0019,circAtlas:hsa-PLS3_0031,Homo sapiens,PLS3,+,ENSG00000102024.17,chrX,115629198,115634081,4,...,1.0,1.0,0.311574,0.741670,0.0,NaN,NaN,NaN,NaN,NaN
10540,TC-hsa-PLS3_0020,circAtlas:hsa-PLS3_0003; circpedia2:HSA_CIRCpe...,Homo sapiens,PLS3,+,ENSG00000102024.17,chrX,115629198,115636978,5,...,1.0,1.0,0.391424,0.790198,0.0,NaN,NaN,NaN,NaN,NaN
10547,TC-hsa-PLS3_0026,circAtlas:hsa-PLS3_0054,Homo sapiens,PLS3,+,ENSG00000102024.17,chrX,115629835,115635046,4,...,1.0,1.0,0.329655,0.812479,0.0,NaN,NaN,NaN,NaN,NaN


In [7]:
ids_not_mapped = df_not_mapped['TransCirc_ID'].to_list()
len(ids_not_mapped)

894

In [8]:
'''
for id_circ in ids_not_mapped:

    command = 'wget https://www.biosino.org/transcirc/detail/' + str(id_circ)

    os.system(command)
'''

"\nfor id_circ in ids_not_mapped:\n\n    command = 'wget https://www.biosino.org/transcirc/detail/' + str(id_circ)\n\n    os.system(command)\n"

In [9]:
pages_path = 'data_raw/TransCirc_web_pages/'

file_names = os.listdir(pages_path)

len(file_names)

894

In [10]:
df_map_2 = pd.DataFrame(columns=['transcirc_id', 'circ_strand', 'circ_chrom', 'tis_coordinate_hg38', 'tis_position_in_circrna'])

for file in file_names:

    file_path = pages_path + file

    with open(file_path, 'r') as f:
        content = f.read()

    strand = df_transcirc[df_transcirc['TransCirc_ID'] == file]['strand'].values
    strand = str(strand[0])

    chrom = df_transcirc[df_transcirc['TransCirc_ID'] == file]['chrom'].values
    chrom = str(chrom[0])

    i = -1
    search = True

    while search:

        i += 1

        pattern_1 = 'id="TIS' + str(i) + '" class="exon" onclick="hightLight2(\''
        pattern_1_new = 'xxxxx'
        new_content = content.replace(pattern_1, pattern_1_new)

        search = re.search(rf'{pattern_1_new}(\d+)', new_content)

        if search:
        
            tis_position = int(search.group(1))

            pattern_2 = pattern_1_new + str(tis_position) + '-3\', this)">'
            pattern_2_new = 'yyyyy'
            new_content_2 = new_content.replace(pattern_2, pattern_2_new)

            search_2 = re.search(rf'{pattern_2_new}(\d+)', new_content_2)
            tis_coord = int(search_2.group(1))

            df_map_2.loc[df_map_2.shape[0]] = [file, strand, chrom, tis_coord, tis_position]

In [11]:
df_map_2

,transcirc_id,circ_strand,circ_chrom,tis_coordinate_hg38,tis_position_in_circrna
0,TC-hsa-GAS2L3_0038,+,chr12,100594905,152
1,TC-hsa-MARCH7_0018,+,chr2,159729023,15
2,TC-hsa-RPS8_0002,+,chr1,44776729,55
3,TC-hsa-SMC1A_0013,-,chrX,53394801,1405
4,TC-hsa-RP11-447L10_0001,-,chr3,196270531,81
...,...,...,...,...,...
940,TC-hsa-KMT5B_0036,-,chr11,68190104,49
941,TC-hsa-TOP2A_0045,-,chr17,40416754,142
942,TC-hsa-RECQL_0005,-,chr12,21499570,46
943,TC-hsa-SET_0008,+,chr9,128693791,372


In [12]:
df_map_2['transcirc_id'].nunique()

894

In [13]:
df_map_all = pd.concat([df_map_1, df_map_2], ignore_index=True)
df_map_all

,transcirc_id,circ_strand,circ_chrom,tis_coordinate_hg38,tis_position_in_circrna
0,TC-hsa-GNB1_0028,-,chr1,1825453,47
1,TC-hsa-GNB1_0038,-,chr1,1825453,47
2,TC-hsa-GNB1_0041,-,chr1,1825453,47
3,TC-hsa-GNB1_0076,-,chr1,1825453,47
4,TC-hsa-GNB1_0065,-,chr1,1825453,47
...,...,...,...,...,...
10631,TC-hsa-KMT5B_0036,-,chr11,68190104,49
10632,TC-hsa-TOP2A_0045,-,chr17,40416754,142
10633,TC-hsa-RECQL_0005,-,chr12,21499570,46
10634,TC-hsa-SET_0008,+,chr9,128693791,372


In [14]:
df_map_all['transcirc_id'].nunique()

9394

In [15]:
df_map_all.to_csv('data_raw/circRNA_TIS_all.tsv', index=False, sep='\t')